MAX

In [1]:
import pandas as pd
import sklearn as sk
import catboost as ctb
import matplotlib as plt
import numpy as np
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split


In [2]:
transactions_path = "C:\codding\Data_Hack\Sber hack\\transactions.csv"

mcc_code_path = "C:\codding\Data_Hack\Sber hack\hse_sber_hack\\mcc_codes.csv"
trans_type_path = "C:\codding\Data_Hack\Sber hack\hse_sber_hack\\trans_types.csv"

train_path = "C:\codding\Data_Hack\Sber hack\hse_sber_hack\\train.csv"
test_path = "C:\codding\Data_Hack\Sber hack\hse_sber_hack\\test.csv"

In [3]:
transactions = pd.read_csv(transactions_path, index_col="client_id")

In [4]:
mcc_code = pd.read_csv(mcc_code_path, sep=';', index_col="mcc_code")
trans_type = pd.read_csv(trans_type_path, sep=';', index_col='trans_type')

train_id = pd.read_csv(train_path, index_col='client_id')
test_id = pd.read_csv(test_path, index_col='client_id')

In [5]:
transactions["term_id"].isna().sum()

1443778

In [6]:
transactions.drop(['term_id'], inplace=True, axis=1)

In [7]:
mcc_code.head()

,mcc_description
mcc_code,
742,Ветеринарные услуги
1711,"Генеральные подрядчики по вентиляции, теплосна..."
1731,Подрядчики по электричеству
1799,"Подрядчики, специализированная торговля — нигд..."
2741,Разнообразные издательства/печатное дело


In [8]:
train_id.drop(["Unnamed: 0"], inplace=True, axis=1)
test_id.drop(["Unnamed: 0"], inplace=True, axis=1)

In [9]:
train_id.head()

,gender
client_id,
c5f18a1d1d83fda62e51e6ae1d7b8911,1
4b6861803556de90249e13d7cf66f0a9,0
7d3f908e1cd1e3624a1445df5e04c4fa,1
40c585a809c1bd17dc6d283d61acd2d2,0
5bd2ff9f6c591353577488b0dfca6dea,0


In [10]:
test_id.head()

""
client_id
002218c52073faaf23bb26df2cffb138
0069d4050c7aeb341317e2e125e8fd2e
0083fb89aea18c408a5534cdd38d7692
014021fa559786c81e9e1d48d378aabb
01759f92152bbb3de79f9493a4a5cd6d


In [11]:
test_id["gender"] = np.NAN
test_id.head()

,gender
client_id,
002218c52073faaf23bb26df2cffb138,NaN
0069d4050c7aeb341317e2e125e8fd2e,NaN
0083fb89aea18c408a5534cdd38d7692,NaN
014021fa559786c81e9e1d48d378aabb,NaN
01759f92152bbb3de79f9493a4a5cd6d,NaN


In [12]:
transactions[['trans_day', 'trans_time']] = transactions['trans_time'].str.split(' ', expand=True)

In [13]:
transactions = transactions[['trans_day', 'trans_time', 'mcc_code', 'trans_type', 'amount', 'trans_city']]
transactions.head()

,trans_day,trans_time,mcc_code,trans_type,amount,trans_city
client_id,,,,,,
d1bbbc9a0e0410d3cf12a3d2f44f3450,35,08:24:41,4829,2370,-1808.56,Tver
81c7349f7bdcb661bc7d5f5f0fdac6aa,351,13:27:15,6011,7010,7954.51,Kazan
fc0c7c937f139bd6d1c5b5d0a0023dd3,150,16:03:59,6011,2010,-12293.56,Khabarovsk
3495487d5beaa5caaa0682fddc38379a,373,11:54:12,6010,7070,49.52,Kaliningrad
07e279042b1da23710fa2f65034d30b9,340,13:11:45,6010,7070,45.29,Saint Petersburg


In [14]:
def sgn_func(x):
    if x > 0:
        return 0
    else:
        return 1

In [143]:
transactions['amount_sign'] = transactions['amount'].map(sgn_func)
transactions['amount'] = transactions['amount'].map(np.abs)
transactions = transactions[['trans_day', 'trans_time', 'mcc_code', 'trans_type', 'amount', 'amount_sign', 'trans_city']]
transactions.head()

KeyboardInterrupt: 

Zero - money recieved

One - money sent

In [14]:
feature_dict = {
    0 : 'Sunday',
    1 : 'Monday',
    2 : 'Tuesday',
    3 : 'Wednesday',
    4 : 'Thursday', 
    5 : 'Friday', 
    6 : 'Saturday',
}
def day_abs_to_day_rel(x):
    return feature_dict[int(x) % 7]

In [15]:
transactions['trans_day'] = transactions['trans_day'].map(day_abs_to_day_rel)
transactions

,trans_day,trans_time,mcc_code,trans_type,amount,trans_city
client_id,,,,,,
d1bbbc9a0e0410d3cf12a3d2f44f3450,Sunday,08:24:41,4829,2370,-1808.56,Tver
81c7349f7bdcb661bc7d5f5f0fdac6aa,Monday,13:27:15,6011,7010,7954.51,Kazan
fc0c7c937f139bd6d1c5b5d0a0023dd3,Wednesday,16:03:59,6011,2010,-12293.56,Khabarovsk
3495487d5beaa5caaa0682fddc38379a,Tuesday,11:54:12,6010,7070,49.52,Kaliningrad
07e279042b1da23710fa2f65034d30b9,Thursday,13:11:45,6010,7070,45.29,Saint Petersburg
...,...,...,...,...,...,...
e91858185ae43cf3970220748893a01b,Saturday,13:45:38,6010,7070,227.40,Penza
429b6c8255f13bdb506fdbd6e342ccb1,Wednesday,19:22:35,4814,1030,-1084.44,Khabarovsk
9443b6a0d948cf368d8d72eb295bdfdf,Saturday,23:59:15,5411,1110,-569.20,Tver


In [16]:
def full_time_to_hour(x):
    lst = list(map(int, x.split(':')))
    
    if lst[1] > 29:
        lst[0] += 1
    if lst[0] == 24:
        lst[0] = 0
    return lst[0]

In [17]:
transactions['trans_time'] = transactions['trans_time'].map(full_time_to_hour)
transactions

,trans_day,trans_time,mcc_code,trans_type,amount,trans_city
client_id,,,,,,
d1bbbc9a0e0410d3cf12a3d2f44f3450,Sunday,8,4829,2370,-1808.56,Tver
81c7349f7bdcb661bc7d5f5f0fdac6aa,Monday,13,6011,7010,7954.51,Kazan
fc0c7c937f139bd6d1c5b5d0a0023dd3,Wednesday,16,6011,2010,-12293.56,Khabarovsk
3495487d5beaa5caaa0682fddc38379a,Tuesday,12,6010,7070,49.52,Kaliningrad
07e279042b1da23710fa2f65034d30b9,Thursday,13,6010,7070,45.29,Saint Petersburg
...,...,...,...,...,...,...
e91858185ae43cf3970220748893a01b,Saturday,14,6010,7070,227.40,Penza
429b6c8255f13bdb506fdbd6e342ccb1,Wednesday,19,4814,1030,-1084.44,Khabarovsk
9443b6a0d948cf368d8d72eb295bdfdf,Saturday,0,5411,1110,-569.20,Tver


In [18]:
train = transactions.merge(right=train_id, on='client_id', how='inner')
train

,trans_day,trans_time,mcc_code,trans_type,amount,trans_city,gender
client_id,,,,,,,
d1bbbc9a0e0410d3cf12a3d2f44f3450,Sunday,8,4829,2370,-1808.56,Tver,0
d1bbbc9a0e0410d3cf12a3d2f44f3450,Sunday,13,4829,2370,-3390.41,Tver,0
d1bbbc9a0e0410d3cf12a3d2f44f3450,Sunday,20,4814,1030,-144.50,Tver,0
d1bbbc9a0e0410d3cf12a3d2f44f3450,Saturday,9,6011,2010,-3542.30,Tver,0
d1bbbc9a0e0410d3cf12a3d2f44f3450,Thursday,14,6011,2010,-3542.70,Tver,0
...,...,...,...,...,...,...,...
b6714c0ca16193cbd419623eb70b1488,Thursday,17,4814,1030,-73.31,Khabarovsk,1
ee4c5d78cbb27771979714ffa903ef5d,Friday,15,6011,7010,72.71,Moscow,0
d428ad799c5790b036beb67cc1b81075,Thursday,17,6011,2010,-14463.42,Khabarovsk,0


In [19]:
test = transactions.merge(right=test_id, on='client_id', how='inner')
test

,trans_day,trans_time,mcc_code,trans_type,amount,trans_city,gender
client_id,,,,,,,
fc0c7c937f139bd6d1c5b5d0a0023dd3,Wednesday,16,6011,2010,-12293.56,Khabarovsk,NaN
fc0c7c937f139bd6d1c5b5d0a0023dd3,Saturday,0,6012,6110,10484.57,Khabarovsk,NaN
fc0c7c937f139bd6d1c5b5d0a0023dd3,Sunday,15,6010,7030,21693.39,Khabarovsk,NaN
fc0c7c937f139bd6d1c5b5d0a0023dd3,Saturday,0,6012,6110,1084.30,Khabarovsk,NaN
fc0c7c937f139bd6d1c5b5d0a0023dd3,Wednesday,18,6011,2010,-72311.38,Khabarovsk,NaN
...,...,...,...,...,...,...,...
06fbfdd5130e86ba92dc8227c9ab576f,Saturday,0,5541,1110,-746.46,Novosibirsk,NaN
756a71cb14c23dbc29b7ed8397dc036b,Wednesday,16,6011,2010,-2168.39,Kazan,NaN
756a71cb14c23dbc29b7ed8397dc036b,Monday,8,4814,1030,-37.15,Kazan,NaN


In [51]:
train.to_csv('C:\codding\Data_Hack\Sber hack\\ready_train.csv')
test.to_csv('C:\codding\Data_Hack\Sber hack\\ready_test.csv')

In [21]:
def features_creation_advanced(x): 
    features = []
    features.append(pd.Series(x[x['amount']>0]['amount'].agg(['min', 'max', 'mean', 'median', 'std', 'count'])\
                                                        .add_prefix('positive_transactions_')))
    features.append(pd.Series(x[x['amount']<0]['amount'].agg(['min', 'max', 'mean', 'median', 'std', 'count'])\
                                                        .add_prefix('negative_transactions_')))
 
    return pd.concat(features)

In [50]:
train.head()

,trans_day,trans_time,mcc_code,trans_type,amount,trans_city,gender
client_id,,,,,,,
d1bbbc9a0e0410d3cf12a3d2f44f3450,Sunday,8,4829,2370,-1808.56,Tver,0
d1bbbc9a0e0410d3cf12a3d2f44f3450,Sunday,13,4829,2370,-3390.41,Tver,0
d1bbbc9a0e0410d3cf12a3d2f44f3450,Sunday,20,4814,1030,-144.50,Tver,0
d1bbbc9a0e0410d3cf12a3d2f44f3450,Saturday,9,6011,2010,-3542.30,Tver,0
d1bbbc9a0e0410d3cf12a3d2f44f3450,Thursday,14,6011,2010,-3542.70,Tver,0


In [23]:
train_agr = train.groupby(train.index).apply(features_creation_advanced)

In [24]:
train_agr

amount,positive_transactions_min,positive_transactions_max,positive_transactions_mean,positive_transactions_median,positive_transactions_std,positive_transactions_count,negative_transactions_min,negative_transactions_max,negative_transactions_mean,negative_transactions_median,negative_transactions_std,negative_transactions_count
client_id,,,,,,,,,,,,
0002cf30347684df542e1a931f356875,722.09,42665.12,18450.900313,16450.650,14360.498086,32.0,-39772.06,-6.13,-1881.786653,-269.295,4111.309461,472.0
0002f4661e0d4d2ae097d78552dad022,312.99,15909.56,2364.986667,723.370,5083.124389,9.0,-9400.52,-6.58,-289.255194,-123.305,693.683636,258.0
00081e0eca8ad94e493691f8f00099c6,360.75,5423.80,1981.352400,1373.800,1704.496840,25.0,-5424.42,-1.89,-379.101205,-144.935,701.946022,166.0
0019bebcb5435df3103a5b0b52f5fc6b,14.93,20247.50,1032.745050,722.430,1743.349204,301.0,-36518.54,-0.32,-743.432256,-361.160,2027.349534,665.0
001c8191ecac499718b33dc293a99e5b,145.62,3616.52,1250.236923,723.710,950.703133,39.0,-15907.79,-11.56,-1394.087964,-178.110,2918.020385,167.0
...,...,...,...,...,...,...,...,...,...,...,...,...
ffe87ae84cf5d83af6a8a5de6dfe327f,11.70,36157.09,4291.427446,2965.470,4821.935240,231.0,-37944.73,-0.05,-2101.189048,-685.150,3526.718762,725.0
fff69a007a119605d6ff25c086840e7d,71.29,10847.20,3621.554167,3614.815,3664.842289,12.0,-10756.68,-7.89,-608.900504,-224.420,1182.793697,337.0
fff78af489ca1024ba0ac7b3c2e47b60,2168.91,7231.79,4283.123077,3616.610,1335.510188,13.0,-36156.56,-93.58,-1932.731049,-145.470,5127.440610,162.0


In [25]:
train_agr = train.merge(right=train_agr, on='client_id', how='inner')
train_agr

,trans_day,trans_time,mcc_code,trans_type,amount,trans_city,gender,positive_transactions_min,positive_transactions_max,positive_transactions_mean,positive_transactions_median,positive_transactions_std,positive_transactions_count,negative_transactions_min,negative_transactions_max,negative_transactions_mean,negative_transactions_median,negative_transactions_std,negative_transactions_count
client_id,,,,,,,,,,,,,,,,,,,
d1bbbc9a0e0410d3cf12a3d2f44f3450,Sunday,8,4829,2370,-1808.56,Tver,0,71.53,36155.75,7020.60375,3001.485,9884.089394,32.0,-144624.04,-3.72,-3998.836936,-891.770,10379.746341,1214.0
d1bbbc9a0e0410d3cf12a3d2f44f3450,Sunday,13,4829,2370,-3390.41,Tver,0,71.53,36155.75,7020.60375,3001.485,9884.089394,32.0,-144624.04,-3.72,-3998.836936,-891.770,10379.746341,1214.0
d1bbbc9a0e0410d3cf12a3d2f44f3450,Sunday,20,4814,1030,-144.50,Tver,0,71.53,36155.75,7020.60375,3001.485,9884.089394,32.0,-144624.04,-3.72,-3998.836936,-891.770,10379.746341,1214.0
d1bbbc9a0e0410d3cf12a3d2f44f3450,Saturday,9,6011,2010,-3542.30,Tver,0,71.53,36155.75,7020.60375,3001.485,9884.089394,32.0,-144624.04,-3.72,-3998.836936,-891.770,10379.746341,1214.0
d1bbbc9a0e0410d3cf12a3d2f44f3450,Thursday,14,6011,2010,-3542.70,Tver,0,71.53,36155.75,7020.60375,3001.485,9884.089394,32.0,-144624.04,-3.72,-3998.836936,-891.770,10379.746341,1214.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
b6714c0ca16193cbd419623eb70b1488,Thursday,17,4814,1030,-73.31,Khabarovsk,1,NaN,NaN,NaN,NaN,NaN,0.0,-73.31,-73.31,-73.310000,-73.310,NaN,1.0
ee4c5d78cbb27771979714ffa903ef5d,Friday,15,6011,7010,72.71,Moscow,0,72.71,72.71,72.71000,72.710,NaN,1.0,NaN,NaN,NaN,NaN,NaN,0.0
d428ad799c5790b036beb67cc1b81075,Thursday,17,6011,2010,-14463.42,Khabarovsk,0,NaN,NaN,NaN,NaN,NaN,0.0,-18078.07,-14463.42,-16270.745000,-16270.745,2555.943527,2.0


In [227]:
train_agr = sk.utils.shuffle(train_agr)

In [229]:
train_agr

,trans_day,trans_time,mcc_code,trans_type,amount,trans_city,gender,positive_transactions_min,positive_transactions_max,positive_transactions_mean,positive_transactions_median,positive_transactions_std,positive_transactions_count,negative_transactions_min,negative_transactions_max,negative_transactions_mean,negative_transactions_median,negative_transactions_std,negative_transactions_count
client_id,,,,,,,,,,,,,,,,,,,
fbf94bb37e250669ea74b20ddaa89e0e,Thursday,8,6010,7070,578.97,Kaliningrad,1,0.02,24808.32,455.016917,285.360,573.925047,83848.0,-144625.32,-0.04,-82719.655484,-72312.890,37220.148265,465.0
91e1f5db3ac7948046fb86211a168dcc,Friday,23,5499,1010,-147.84,Vladivostok,1,0.15,7232.24,2011.467234,1085.570,2047.961859,47.0,-19030.16,-1.23,-488.123985,-181.880,1238.673832,1079.0
fd80e3e70cc59bccc60eaaeeae43a748,Tuesday,11,4814,1030,-57.30,Vladimir,1,360.65,11858.94,4310.027000,2531.470,4535.100799,10.0,-23140.16,-2.55,-993.205255,-103.910,2170.567182,411.0
93bb501f6f04706717c691bad7d0c947,Tuesday,0,6011,7010,723.27,Moscow,1,37.08,82435.22,4952.034808,1085.450,11983.381065,52.0,-57849.19,-0.55,-882.403829,-217.055,3749.780120,316.0
d3c363ead85a10db0ab3377bf0b9e722,Saturday,14,8021,1010,-7633.22,Saint Petersburg,0,2.38,36156.46,5527.490256,1633.430,7967.803035,39.0,-24585.44,-3.74,-651.265261,-107.910,2256.235620,555.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
c2b926e069f9ef39fd487e0fa31f8d3b,Sunday,18,7542,1110,-2168.80,Kaliningrad,0,36.10,65081.54,5507.431111,1482.365,11399.086374,36.0,-36155.07,-35.12,-1685.566007,-587.150,3539.798180,303.0
b3110f6506703cd8dfe897f14a7678bc,Sunday,17,4829,4071,-71.31,Moscow,0,0.18,361561.48,38094.245688,7365.205,72753.367714,378.0,-361561.60,-0.11,-3441.377885,-431.530,10943.716135,4047.0
cd1052d1a45ad8666733c046e533f55c,Monday,5,4814,1030,-36.59,Vladivostok,0,13.76,12292.61,1008.843151,722.605,1295.291021,146.0,-12220.17,-0.97,-369.975332,-115.975,826.899929,392.0


In [26]:
train_new_agr = train_agr[:700000]

In [27]:
y = train_new_agr['gender']
X = train_new_agr.drop(['gender'], axis=1)
X = pd.get_dummies(X, columns=['mcc_code', 'trans_type', 'trans_day', 'trans_city'], dtype=int, drop_first=True)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)

In [29]:
X_train

,trans_time,amount,positive_transactions_min,positive_transactions_max,positive_transactions_mean,positive_transactions_median,positive_transactions_std,positive_transactions_count,negative_transactions_min,negative_transactions_max,...,trans_day_Wednesday,trans_city_Kazan,trans_city_Khabarovsk,trans_city_Moscow,trans_city_Novosibirsk,trans_city_Penza,trans_city_Saint Petersburg,trans_city_Tver,trans_city_Vladimir,trans_city_Vladivostok
client_id,,,,,,,,,,,,,,,,,,,,,
8c8844c1a37fb0c87a5081c8bd49fd38,17,-88.77,1.23,21694.66,1651.474421,360.560,2904.816810,233.0,-23862.64,-0.09,...,0,0,0,0,0,0,0,0,0,1
97cff2ce09a64d9c31cc9afe2dca61ba,7,722.97,7.84,216935.84,1907.551961,362.150,8832.280181,1627.0,-94063.88,-6.57,...,0,0,0,0,0,0,1,0,0,0
fbf94bb37e250669ea74b20ddaa89e0e,17,658.93,0.02,24808.32,455.016917,285.360,573.925047,83848.0,-144625.32,-0.04,...,0,0,0,0,0,0,0,0,0,0
210a288dba0bd3bf6ce90f1f1b188951,21,4275.97,1.21,216937.63,4705.077219,2541.375,13122.105812,12904.0,-216937.59,-1.11,...,0,0,0,0,0,0,1,0,0,0
7e8ed6ae17e9907c1dcfc2cfbcfe27dd,21,-7954.41,107.81,11570.40,2281.039268,1663.230,2504.230121,41.0,-14462.83,-0.67,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5fb513874d1d93f030b610b5cf8b36c1,0,-136.01,0.26,418687.41,32810.967609,5784.815,72260.622657,92.0,-180780.59,-3.83,...,0,0,0,0,0,0,0,1,0,0
fbf94bb37e250669ea74b20ddaa89e0e,13,144.28,0.02,24808.32,455.016917,285.360,573.925047,83848.0,-144625.32,-0.04,...,1,0,0,0,0,0,0,0,0,0
0a8597548725cb2cdcc28f525159f035,0,-325.69,72.33,37963.87,3514.742222,1988.545,6716.924279,54.0,-24586.41,-1.38,...,0,1,0,0,0,0,0,0,0,0


In [32]:
test_model = ctb.CatBoostClassifier(
            eval_metric='AUC:hints=skip_train~false',
            loss_function='Logloss',
            auto_class_weights='Balanced',
            verbose=100,
            iterations=150,
            learning_rate=0.05
)


In [33]:
test_model.fit(X=X_train, y=y_train)

0:	learn: 0.8600239	total: 91ms	remaining: 13.6s
100:	learn: 0.9999929	total: 9.02s	remaining: 4.37s
149:	learn: 1.0000000	total: 13.3s	remaining: 0us


In [34]:
test_output = test_model.predict(X_test)

In [35]:
test_output

array([0, 1, 1, ..., 0, 0, 0], dtype=int64)

In [37]:
X_test_new = X_test
X_test_new['test_output'] = test_output
X_test_new["real_output"] = y_test

X_test_new

,trans_time,amount,positive_transactions_min,positive_transactions_max,positive_transactions_mean,positive_transactions_median,positive_transactions_std,positive_transactions_count,negative_transactions_min,negative_transactions_max,...,trans_city_Khabarovsk,trans_city_Moscow,trans_city_Novosibirsk,trans_city_Penza,trans_city_Saint Petersburg,trans_city_Tver,trans_city_Vladimir,trans_city_Vladivostok,test_output,real_output
client_id,,,,,,,,,,,,,,,,,,,,,
06de099360cb07372274ad6a60ffc03f,16,811.31,3.96,115979.65,4308.021532,1012.180,12556.033098,1997.0,-108468.99,-2.85,...,0,0,0,0,0,0,0,0,0,0
4767b939762b8356d699337b3cfa6b68,21,-106.13,2.72,18801.84,3786.765376,2530.550,4049.170724,173.0,-72312.39,-0.43,...,0,0,1,0,0,0,0,0,1,1
b1b185b585a46add4b641a6abd75dda6,11,-2891.78,71.34,68695.56,12444.797083,7231.770,14593.986163,48.0,-72120.65,-6.22,...,0,1,0,0,0,0,0,0,1,1
00a258324f4699f3b033aa627f39ec3c,9,-14461.84,35.72,159087.19,16393.664118,2459.500,38296.402520,17.0,-65436.00,-3.23,...,1,0,0,0,0,0,0,0,1,1
faa54181412b23927ce4efb578205fb6,18,-577.43,360.77,7230.67,3388.010000,3615.290,1854.815194,20.0,-18728.28,-10.59,...,0,0,1,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79cefee72e96c31db6c7083bcd942e62,17,-264.77,109.23,288453.41,7949.157818,724.120,39025.638605,55.0,-108468.70,-4.79,...,0,0,0,1,0,0,0,0,1,1
fbf94bb37e250669ea74b20ddaa89e0e,5,323.81,0.02,24808.32,455.016917,285.360,573.925047,83848.0,-144625.32,-0.04,...,0,0,0,0,0,0,0,0,1,1
7fe6af0e205e2d404cabfaa3d726ab32,18,-154.80,49.91,126546.91,12367.911289,3616.295,19505.990158,194.0,-130162.74,-2.28,...,0,0,0,0,1,0,0,0,0,0


In [47]:
somename = X_test_new.groupby(X_test_new.index)['test_output'].agg(pd.Series.mode)
somename2 = X_test_new.groupby(X_test_new.index)['real_output'].agg(pd.Series.mode)

somename2

client_id
0002cf30347684df542e1a931f356875    0
004a16067342f723eb103dc7d7e4eb94    0
00a258324f4699f3b033aa627f39ec3c    1
025844b79172c2188ca7356d0b2f01bc    0
0287cebd29224d63e4df36aba749be25    0
                                   ..
ff5274c2b227d6fb7ab2a878ac995040    0
ff7d857efcb99e10578916d83f5a9c10    0
ffdfe7ace44cae54b6ec9be8383d69da    0
ffe87ae84cf5d83af6a8a5de6dfe327f    0
fff69a007a119605d6ff25c086840e7d    0
Name: real_output, Length: 683, dtype: int64

In [49]:
somename

client_id
0002cf30347684df542e1a931f356875    0
004a16067342f723eb103dc7d7e4eb94    0
00a258324f4699f3b033aa627f39ec3c    1
025844b79172c2188ca7356d0b2f01bc    0
0287cebd29224d63e4df36aba749be25    0
                                   ..
ff5274c2b227d6fb7ab2a878ac995040    0
ff7d857efcb99e10578916d83f5a9c10    0
ffdfe7ace44cae54b6ec9be8383d69da    0
ffe87ae84cf5d83af6a8a5de6dfe327f    0
fff69a007a119605d6ff25c086840e7d    0
Name: test_output, Length: 683, dtype: int64

In [48]:
metrics.roc_auc_score(somename, somename2)

0.9986631016042781